**Waste Classification Recyclable or Organic Product**


In [ ]:
#Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Importing libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import keras

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

In [ ]:
%cd /content/drive/My Drive/waste_classification

/content/drive/My Drive/waste_classification


In [ ]:
#Initialising the variable
train_path = '/content/drive/My Drive/waste_classification/dataset/training_set'
test_path = '/content/drive/My Drive/waste_classification/dataset/test_set'
img_width = 64
img_height = 64
no_of_output = 2


### #Part 1 - Building the CNN

In [ ]:
#Part 1 - Building the CNN

#Function for creating the architecture of model
def cnn_model():
    
    #Initialising the cnn
    classifier = Sequential()
    
    #Adding Convolution Layer
    classifier.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
    classifier.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    classifier.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    #Maxpooling
    classifier.add(MaxPooling2D(pool_size=(2, 2)))
    #Adding Dropout to prevent form overfitting
    classifier.add(Dropout(0.25))

    classifier.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    classifier.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    classifier.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    
    classifier.add(MaxPooling2D(pool_size=(2, 2)))
    classifier.add(Dropout(0.25))
    
    #Flattening of data
    classifier.add(Flatten())

    #Adding full connection layer
    classifier.add(Dense(256, activation='relu'))
    classifier.add(Dropout(0.25))
    classifier.add(Dense(256, activation='relu'))
    classifier.add(Dropout(0.25))
    
    #Adding Output layer
    classifier.add(Dense(no_of_output, activation='softmax'))

    #compiling the model
    classifier.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
              
    return classifier



In [ ]:
classifier= cnn_model()
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 60, 32)        9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 58, 58, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 29, 29, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 29, 29, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 27, 27, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 25, 25, 64)        3

In [ ]:
checkpoint_path= "/content/drive/My Drive/waste_classification/model/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

check_point = ModelCheckpoint(checkpoint_path,  
                             monitor='val_acc', 
                             save_best_only=True
                             )


In [ ]:
#Function for predicting the input image
def use_model(path):
    test_image = image.load_img(path, target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    classes = classifier.predict_classes(test_image)
    return classes

### #Image Data Preprocessing

In [ ]:
#Image Data Preprocessing
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (img_height,img_width),
                                                 classes = ['O','R'])

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (img_height,img_width),
                                            classes = ['O','R'])

Found 9427 images belonging to 2 classes.
Found 1523 images belonging to 2 classes.


In [ ]:
!pwd

/content/drive/My Drive/waste_classification


In [ ]:
#run code to train the neural network

classifier.fit(training_set,  
                             validation_data= test_set,  
                             epochs= 100, 
                             callbacks=[check_point])

classifier.save('my_model.h5')

Epoch 1/100
295/295 [==============================] - 2161s 7s/step - loss: 0.5205 - accuracy: 0.7502 - val_loss: 0.4619 - val_accuracy: 0.7873
Epoch 2/100
295/295 [==============================] - 33s 113ms/step - loss: 0.4418 - accuracy: 0.8061 - val_loss: 0.4666 - val_accuracy: 0.8148
Epoch 3/100
295/295 [==============================] - 32s 110ms/step - loss: 0.4167 - accuracy: 0.8235 - val_loss: 0.3634 - val_accuracy: 0.8582
Epoch 4/100
295/295 [==============================] - 33s 110ms/step - loss: 0.3897 - accuracy: 0.8314 - val_loss: 0.3929 - val_accuracy: 0.8523
Epoch 5/100
295/295 [==============================] - 34s 114ms/step - loss: 0.3806 - accuracy: 0.8395 - val_loss: 0.3081 - val_accuracy: 0.8798
Epoch 6/100
295/295 [==============================] - 33s 113ms/step - loss: 0.3628 - accuracy: 0.8501 - val_loss: 0.3135 - val_accuracy: 0.8825
Epoch 7/100
295/295 [==============================] - 33s 113ms/step - loss: 0.3636 - accuracy: 0.8496 - val_loss: 0.3794 - 